* if the feature are category feature, you can do this easily. calculate each class's target mean and use the mean as a column!

* http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/target-encoding.html

In [ ]:
                                                                                                                                                # :kaggle: 0.899
# :here: 0.8971399999999999


# best so far:
# params = {
#     'min_child_weight': 10.0,
#     'objective': 'binary:logistic',
#     'max_depth': 3,
#     'max_delta_step': 1.8,
#     'colsample_bytree': 0.4,
#     'subsample': 0.8,
#     'eta': 0.05,
#     'gamma': 0.65,
#     'num_boost_round' : 700,
#     'tree_method':'approx',
#      "eval_metric": "auc"
#     }

# for i, (train_index, test_index) in enumerate(skf.split(X, y)):
#     print('[Fold %d/%d]' % (i + 1, kfold))
#     X_train, X_valid = X[train_index], X[test_index]
#     y_train, y_valid = y[train_index], y[test_index]
#     # Convert our data into XGBoost format
#     d_train = xgb.DMatrix(X_train, y_train)
#     d_valid = xgb.DMatrix(X_valid, y_valid)
#     d_test = xgb.DMatrix(test.values)
#     watchlist = [(d_train, 'train'), (d_valid, 'valid')]
#     mdl = xgb.train(params, d_train, 3000, watchlist, early_stopping_rounds=90, maximize=True, verbose_eval=100)

#     print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
#     # Predict on our test data
#     p_test = mdl.predict(d_test, ntree_limit=mdl.best_ntree_limit)
#     sub['target'] += p_test/kfold

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


In [2]:
# train = pd.read_csv('train_NEWFEATURES.csv')
# train = train.drop(['Unnamed: 0'],axis=1)
# test = pd.read_csv('test_NEWFEATURES.csv')
# test = test.drop(['Unnamed: 0'],axis=1)

In [3]:
train = pd.read_csv('data/santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('data/santander-customer-transaction-prediction/test.csv')

In [23]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]
for feature in features:
    train['r2_'+feature] = np.round(train[feature], 2)
    test['r2_'+feature] = np.round(test[feature], 2)
    train['r1_'+feature] = np.round(train[feature], 1)
    test['r1_'+feature] = np.round(test[feature], 1)

In [24]:
y = train.target
X = train.drop(['ID_code', 'target'], axis=1)
X_test = test.drop(['ID_code'], axis=1)

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [19]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]

In [26]:
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)
d_test = xgb.DMatrix(X_test)

### build a benchmark

1. selection of features

2. chossing performance metric

3. choosing a classifier and optimization algorithm

4. evaluate the performance of the model

5. tuning the algorithm

*Area under receiver operating curve*


## Perceptron

In [160]:
ppn = Perceptron(max_iter=5000, eta0=0.01, random_state=1)
ppn.fit(X_train_std, y_train)

In [ ]:
y_pred = ppn.predict(X_valid_std)

In [ ]:
roc_auc_score(y_valid, y_pred)

In [15]:
forest = RandomForestClassifier(criterion='gini',
                                n_estimators=150, 
                                random_state=1,
                                n_jobs=-1)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [16]:
y_pred = forest.predict(X_valid)
roc_auc_score(y_valid, y_pred)

0.5

In [ ]:
x =10
print(x)

In [14]:
watchlist = [(d_train,'train'), (d_valid,'valid')]

In [21]:
params = {
    'boosting':'dart',
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 3,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.5,
    'subsample': 0.8,
    'eta': 0.045,
    'gamma': 0.65,
    'num_boost_round' : 700,
    'tree_method':'approx',
     "eval_metric": "auc"
    }

mdl = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=100, maximize=True, verbose_eval=100)

[0]	train-auc:0.581086	valid-auc:0.573249
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 100 rounds.
[100]	train-auc:0.808019	valid-auc:0.7924
[200]	train-auc:0.853744	valid-auc:0.832053
[300]	train-auc:0.875797	valid-auc:0.850153
[400]	train-auc:0.889721	valid-auc:0.861636
[500]	train-auc:0.899114	valid-auc:0.869153
[600]	train-auc:0.906544	valid-auc:0.875061
[700]	train-auc:0.912194	valid-auc:0.878935
[800]	train-auc:0.91696	valid-auc:0.882651
[900]	train-auc:0.920868	valid-auc:0.885173
[999]	train-auc:0.924322	valid-auc:0.887392


### Factorization Machines are probably a great candidate for this
* https://www.analyticsvidhya.com/blog/2018/01/factorization-machines/

In [22]:
columnId = 'ID_code'
columnTarget = 'target'
sub = pd.DataFrame(test[columnId], columns=[columnId,columnTarget])
sub.head()

,ID_code,target
0,test_0,NaN
1,test_1,NaN
2,test_2,NaN
3,test_3,NaN
4,test_4,NaN


In [23]:
pred = mdl.predict(d_test)

In [24]:
sub[columnTarget]=pred

In [25]:
sub.head()

,ID_code,target
0,test_0,0.085930
1,test_1,0.245788
2,test_2,0.157744
3,test_3,0.145868
4,test_4,0.044896


In [38]:
model.__class__.__name__

'Booster'

In [26]:
sub.to_csv('submitPR_{}.csv'.format(mdl.__class__.__name__), index=False)

In [57]:
def submissionFile(clf):
    print('Creating submission file')
    pred = clf.predict(features) 
    sub[columnTarget]=pred
    print('submit_{}.csv'.format(clf.__class__.__name__))
    sub.to_csv('submit_{}.csv'.format(clf.__class__.__name__), index=False)

In [137]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
from sklearn.model_selection import StratifiedKFold

In [27]:
# https://www.kaggle.com/sudosudoohio/stratified-kfold-xgboost-eda-tutorial-0-281
X = train.drop(['ID_code', 'target'], axis=1).values
X_for_aug = train
y = train.target.values
X_test = test.drop(['ID_code'], axis=1)
test_id = test.ID_code.values

In [28]:
X_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,r2_var_195,r1_var_195,r2_var_196,r1_var_196,r2_var_197,r1_var_197,r2_var_198,r1_var_198,r2_var_199,r1_var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,2.47,2.5,4.37,4.4,10.72,10.7,15.47,15.5,-8.72,-8.7
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,0.48,0.5,-1.49,-1.5,9.87,9.9,19.13,19.1,-20.98,-21.0
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,2.13,2.1,-7.11,-7.1,7.06,7.1,19.90,19.9,-23.18,-23.2
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,3.17,3.2,3.96,4.0,9.23,9.2,13.02,13.0,-4.21,-4.2
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,-0.29,-0.3,-5.16,-5.2,7.29,7.3,13.93,13.9,-9.18,-9.2


In [31]:
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)

In [38]:
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 5,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.2,
    'subsample': 0.6,
    'eta': 0.01,
    'gamma': 0.65,
    'num_boost_round' : 700,
    'tree_method':'approx',
     "eval_metric": "auc"
    }

In [39]:
## Inspiration from
#https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
def augment(train,num_n=1,num_p=2):
    newtrain=[train]
    
    n=train[train.target==0]
    for i in range(num_n):
        newtrain.append(n.apply(lambda x:x.values.take(np.random.permutation(len(n)))))
    
    for i in range(num_p):
        p=train[train.target>0]
        newtrain.append(p.apply(lambda x:x.values.take(np.random.permutation(len(p)))))
    return pd.concat(newtrain)
#df=oversample(train,2,1)

In [40]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = np.zeros_like(test_id)

In [41]:
sub.head()

,id,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [43]:
X_test.shape

(200000, 600)

In [44]:
for i, (train_index, test_index) in enumerate(skf.split(train, y)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    
    t = train.iloc[train_index]
    t = augment(t)   
    train_data = t.drop(['target','ID_code'], axis=1)
    y_train = t.target.values
    
    
    valid_data= train.iloc[test_index]
    y_valid = valid_data.target.values
    valid_data = valid_data.drop(['target','ID_code'], axis=1)
   
   
    # Convert our data into XGBoost format
    d_train = xgb.DMatrix(train_data, y_train)
    d_valid = xgb.DMatrix(valid_data, y_valid)
    d_test = xgb.DMatrix(X_test)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    mdl = xgb.train(params, d_train, 15000, watchlist, early_stopping_rounds=300, maximize=True, verbose_eval=100)

    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    p_test = mdl.predict(d_test, ntree_limit=mdl.best_ntree_limit)
    sub['target'] += p_test/kfold

[Fold 1/5]
[0]	train-auc:0.626438	valid-auc:0.617105
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 300 rounds.
[100]	train-auc:0.807371	valid-auc:0.721957
[200]	train-auc:0.835762	valid-auc:0.74569
[300]	train-auc:0.857091	valid-auc:0.765577
[400]	train-auc:0.876367	valid-auc:0.786151
[500]	train-auc:0.889724	valid-auc:0.800329
[600]	train-auc:0.89985	valid-auc:0.811542
[700]	train-auc:0.907293	valid-auc:0.819928
[800]	train-auc:0.914131	valid-auc:0.827884
[900]	train-auc:0.919331	valid-auc:0.833893
[1000]	train-auc:0.923953	valid-auc:0.839344
[1100]	train-auc:0.927784	valid-auc:0.843478
[1200]	train-auc:0.931429	valid-auc:0.847589
[1300]	train-auc:0.93457	valid-auc:0.850854
[1400]	train-auc:0.937605	valid-auc:0.853948
[1500]	train-auc:0.940198	valid-auc:0.856625
[1600]	train-auc:0.942598	valid-auc:0.858918
[1700]	train-auc:0.944577	valid-auc:0.861067
[1800]	train-auc:0.946619	valid-auc:0.86315
[1900]

In [49]:
sub.to_csv('StratifiedKFoldaugFeatures.csv', index=False)

In [47]:
sub.columns = ["ID_code","target"]

In [46]:
sub.shape

(200000, 2)

In [48]:
sub.head()

,ID_code,target
0,test_0,0.0565045
1,test_1,0.221241
2,test_2,0.216036
3,test_3,0.215526
4,test_4,0.0177518


In [101]:
sub.to_csv("StratKFoldAugmentation.csv", index = None)

### Voting Ensemble

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [249]:
from sklearn.ensemble import VotingClassifier
from sklearn import linear_model
from xgboost import XGBClassifier

In [228]:
model_lm = linear_model.LogisticRegression(penalty="l2", C = 100)

In [229]:
model_xgb = XGBClassifier(missing=np.nan, max_depth=3, n_estimators=2000,
                        min_child_weight = 3,
                        learning_rate=0.02, subsample=0.7, colsample_bytree=0.7, random_state=6744, verbose = 100)

In [247]:
voting_model = VotingClassifier(estimators=[('xgb1', model_xgb),
                                    ('logistic_reg', model_lm)], voting='soft',weights=[8,1])

In [139]:
voting_model_tr = unbooked_gam.fit(X, y)

In [232]:
voting_probas = voting_model_tr.predict_proba(X_predictions)[:,1]

In [234]:
columnId = 'ID_code'
columnTarget = 'target'
sub = pd.DataFrame(inputTest[columnId], columns=[columnId,columnTarget])


,ID_code,target
0,test_0,NaN
1,test_1,NaN
2,test_2,NaN
3,test_3,NaN
4,test_4,NaN


In [18]:
sub.head

NameError: name 'sub' is not defined

In [245]:
sub.to_csv('votingClass.csv', index=None)

In [236]:
sub['target']=voting_probas

In [214]:
roc_auc_score(y_valid, voting_probas)

0.8963790671353871

In [218]:
X_predictions.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [309]:
! pip install lightgbm

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
    100% |████████████████████████████████| 645kB 4.1MB/s 


In [48]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
import lightgbm as lgb
import pandas as pd
import numpy as np

from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold


/Users/jpoberhauser/Desktop/competitions/santander/venv/lib/python2.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [10]:
train=pd.read_csv("data/santander-customer-transaction-prediction/train.csv").drop("ID_code",axis=1)
test=pd.read_csv("data/santander-customer-transaction-prediction/test.csv").drop("ID_code",axis=1)

In [11]:
test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [12]:
## Inspiration from
#https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
def augment(train,num_n=1,num_p=3):
    newtrain=[train]
    
    n=train[train.target==0]
    for i in range(num_n):
        newtrain.append(n.apply(lambda x:x.values.take(np.random.permutation(len(n)))))
    
    for i in range(num_p):
        p=train[train.target>0]
        newtrain.append(p.apply(lambda x:x.values.take(np.random.permutation(len(p)))))
    return pd.concat(newtrain)
#df=oversample(train,2,1)

In [13]:
param = {
   "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "bagging_seed" : 10,
    "verbosity" : 1,
}

In [14]:
result=np.zeros(test.shape[0])

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2,random_state=10)
for counter,(train_index, valid_index) in enumerate(rskf.split(train, train.target),1):
    print (counter)
    
    #Train data
    t=train.iloc[train_index]
    t=augment(t)
    trn_data = lgb.Dataset(t.drop("target",axis=1), label=t.target)
    
    #Validation data
    v=train.iloc[valid_index]
    val_data = lgb.Dataset(v.drop("target",axis=1), label=v.target)
    
    #Training
    model = lgb.train(param, trn_data, 1000000, valid_sets = [val_data], verbose_eval=500, early_stopping_rounds = 3000)
    result += model.predict(test)

1
Training until validation scores don't improve for 3000 rounds.
[500]	valid_0's auc: 0.872273
[1000]	valid_0's auc: 0.879627
[1500]	valid_0's auc: 0.88364
[2000]	valid_0's auc: 0.887469
[2500]	valid_0's auc: 0.8899
[3000]	valid_0's auc: 0.892074
[3500]	valid_0's auc: 0.893635
[4000]	valid_0's auc: 0.894913
[4500]	valid_0's auc: 0.895861
[5000]	valid_0's auc: 0.896565
[5500]	valid_0's auc: 0.897044
[6000]	valid_0's auc: 0.897407
[6500]	valid_0's auc: 0.897603
[7000]	valid_0's auc: 0.897816
[7500]	valid_0's auc: 0.898017
[8000]	valid_0's auc: 0.898056
[8500]	valid_0's auc: 0.898086
[9000]	valid_0's auc: 0.898101
[9500]	valid_0's auc: 0.898151
[10000]	valid_0's auc: 0.898103
[10500]	valid_0's auc: 0.898033
[11000]	valid_0's auc: 0.897971
[11500]	valid_0's auc: 0.897886
[12000]	valid_0's auc: 0.897822
Early stopping, best iteration is:
[9413]	valid_0's auc: 0.898161
2
Training until validation scores don't improve for 3000 rounds.
[500]	valid_0's auc: 0.874705
[1000]	valid_0's auc: 0.882

In [46]:
submission = pd.read_csv('data/santander-customer-transaction-prediction/sample_submission.csv')

In [47]:
submission['target'] = result/counter

In [48]:
submission.head()

,ID_code,target
0,test_0,0.078167
1,test_1,0.223959
2,test_2,0.151597
3,test_3,0.206112
4,test_4,0.021324


In [49]:
submission.to_csv("LightGBM_1NoDataAugExtraFeatFEWERCV.csv", index=False)

In [ ]:
submission = pd.read_csv(path/'sample_submission.csv')

filename="{:%Y-%m-%d_%H_%M}_sub.csv".format(datetime.now())
submission.to_csv(filename, index=False)

In [37]:
train.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,r2_var_195,r1_var_195,r2_var_196,r1_var_196,r2_var_197,r1_var_197,r2_var_198,r1_var_198,r2_var_199,r1_var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,-2.40,-2.4,7.88,7.9,8.56,8.6,12.78,12.8,-1.09,-1.1
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,2.03,2.0,8.13,8.1,8.79,8.8,18.36,18.4,1.95,2.0
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,3.14,3.1,-6.52,-6.5,8.27,8.3,14.72,14.7,0.40,0.4
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,-1.27,-1.3,-2.93,-2.9,10.29,10.3,17.97,18.0,-9.00,-9.0
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.51,-1.5,3.93,3.9,9.50,9.5,18.00,18.0,-8.81,-8.8
